# OCR with EasyOCR


In [2]:
import easyocr  # For OCR
from util import to_data_path

reader = easyocr.Reader(["de"])
transcription_easyocr = reader.readtext(to_data_path("contract1_1.jpg"))

# Print transcription
text = " ".join(map(lambda t: t[1], transcription_easyocr))
print("Transcription (EasyOCR):", text)

Transcription (EasyOCR): $ 2 MietzeitlKündigung 1 Das Mietverhältnis beginnt am 0443.Q023 und wird auf unbestimmte Zeit geschlossen. und wird auf die Dauer von Jahren, also bis geschlossen; weil der Vermieter die Räume nach Ablauf der Mietzeit als Wohnung für sich, seine Familienangehörigen oder Angehörige seines Haushalts nutzen will, hier: in zulässiger Weise beseitigen oder s0 wesentlich verändern oder instand setzen will; dass die Maßnahme durch Fortsetzung des Mietverhältnisses erheblich erschwert werden würde, hier: an einen zur Dienstleistung Verpflichteten vermieten will. Besteht kein solcher Befristungsgrund oder wurde dieser vorstehend nicht schriftlich festgehalten; gilt das Mietver- hältnis als auf unbestimmte Zeit geschlossen. Auch während der Laufzeit der Befristung ist der Vermieter berechtigt; Mieterhöhungen vorzunehmen; die aufgrund gesetzlicher Vorschriften für Wohnraum; der auf unbestimmte Zeit ver - mietet ist, zulässig sind. Beide Parteien können eine ordentliche K

As can be seen, the library struggles with the handwritten parts of the contract.
Thus it's not a suitable solution for this task.

What is needed is a library that also can handle handwritten text (HTR).
For this, after a quick search,
I found an article from HuggingFace presenting HTRflow: <https://huggingface.co/blog/Gabriel/htrflow>

Let's copy their code and adapt it to our needs:


In [4]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from util import get_pil_contract

# Handwriting-specific model and processor
trocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
trocr_model = VisionEncoderDecoderModel.from_pretrained(
    "microsoft/trocr-base-handwritten"
)

handwritten_image = get_pil_contract("1", "0").convert("RGB")

pixel_values = trocr_processor(
    images=handwritten_image, return_tensors="pt"
).pixel_values
generated_ids = trocr_model.generate(pixel_values)
handwritten_text = trocr_processor.batch_decode(
    generated_ids, skip_special_tokens=True
)[0]
print(f"Handwritten text:", handwritten_text)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.47.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

Handwritten text: 2.


The results are underwhelming.
I haven't investigated yet into the way the model way trained, but I think it has to something with the size of the image.
While normally you would have snippets of handwritten parts,
in this case I just fed it the complete page which has a size of 1653x2339 pixels.

Instead of giving it the complete image, why not feed it with rows of the text.
